# Unit 5 - Example 16: GPU Workflows & RAPIDS

## 📚 Learning Objectives

By completing this notebook, you will:
- Understand the key concepts of this topic
- Apply the topic using Python code examples
- Practice with small, realistic datasets or scenarios

## 🔗 Prerequisites

- ✅ Basic Python
- ✅ Basic NumPy/Pandas (when applicable)

---

## Official Structure Reference

This notebook supports **Course 05, Unit 5** requirements from `DETAILED_UNIT_DESCRIPTIONS.md`.

---


# Unit 5 - Example 16: GPU Workflows & RAPIDS

## 🔗 Building on Examples 14 & 15 | البناء على المثالين 14 و 15

**From Example 14 (Dask) & Example 15 (PySpark):**
- We learned distributed computing with Dask and PySpark
- Both are great for CPU-based distributed processing
- But for GPU acceleration, we need RAPIDS

**This notebook solves that problem!**
- We'll learn **RAPIDS** - GPU-accelerated data science ecosystem
- We'll see **complete workflows** on GPU (cuDF, cuML)
- We'll understand **GPU-accelerated data science** pipelines

**This adds GPU acceleration to our distributed computing toolkit!**

In [1]:
# 🚀 Google Colab Setup (Run this first if using Colab)
# دليل إعداد Google Colab (قم بتشغيل هذا أولاً إذا كنت تستخدم Colab)

import os

# Check if running on Colab
try:
    IN_COLAB = 'google.colab' in str(get_ipython())
except NameError:
    IN_COLAB = False

if IN_COLAB:
    print("🌐 Detected Google Colab environment")
    print("📋 To enable GPU:")
    print("   1. Click: Runtime → Change runtime type")
    print("   2. Set Hardware accelerator: GPU")
    print("   3. Click Save")
    print("\n⏳ Installing RAPIDS for GPU acceleration...")
    print("   (This may take 5-10 minutes)")
    
    # Install RAPIDS
    !pip install -q cudf-cu11 cuml-cu11 --extra-index-url=https://pypi.nvidia.com
    
    print("\n✅ RAPIDS installed!")
    print("🔄 Please restart runtime: Runtime → Restart runtime")
    print("   Then run this cell again to verify installation.")
else:
    print("💻 Running on local machine")
    print("   For Colab setup, see: DOCS/COLAB_SETUP.md")

💻 Running on local machine
   For Colab setup, see: DOCS/COLAB_SETUP.md


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from sklearn.linear_model import LinearRegression as skLinearRegression


In [3]:
# Try to import RAPIDS
try:
    import cudf
    import cuml
    from cuml.linear_model import LinearRegression
    CUDF_AVAILABLE = True
    RAPIDS_AVAILABLE = True  # For RAPIDS-specific features
    print("✓ RAPIDS is available")
except ImportError:
    CUDF_AVAILABLE = False
    RAPIDS_AVAILABLE = False
    print("⚠ RAPIDS not available - Using simulation")

print("=" * 70)
print("Example 16: GPU Workflows & RAPIDS | سير عمل GPU و RAPIDS")
print("=" * 70)
print("\n📚 Prerequisites: Example 14 completed, basic GPU knowledge")
print("🔗 This is the SECOND example in Unit 5 - GPU workflows")
print("🎯 Goal: Master GPU-accelerated data science with RAPIDS")


⚠ RAPIDS not available - Using simulation
Example 16: GPU Workflows & RAPIDS | سير عمل GPU و RAPIDS

📚 Prerequisites: Example 14 completed, basic GPU knowledge
🔗 This is the SECOND example in Unit 5 - GPU workflows
🎯 Goal: Master GPU-accelerated data science with RAPIDS


In [4]:
# 1. DATA PROCESSING WORKFLOW


In [5]:
print("\n1. GPU Data Processing Workflow")
print("-" * 70)
np.random.seed(42)
n_samples = 500000
data = {
'id': range(n_samples), 'value1': np.random.randn(n_samples),
'value2': np.random.randn(n_samples), 'category': np.random.choice(['A', 'B', 'C'], n_samples),
'score': np.random.randint(0, 100, n_samples)
}
df_pandas = pd.DataFrame(data)
if RAPIDS_AVAILABLE:
    print("\nUsing cuDF (GPU)...")
    df_gpu = cudf.DataFrame(data)
    # GPU operations
    start_time = time.time()
    result_gpu = df_gpu.groupby('category').agg({
        'value1': 'mean', 'value2': 'std',
        'score': ['mean', 'max']
    })
    gpu_time = time.time() - start_time
    print(f"GPU processing time: {gpu_time:.4f} seconds")
else:
    print("\n⚠ Simulating GPU workflow with pandas")
    df_gpu = df_pandas.copy()
    start_time = time.time()
    result_gpu = df_gpu.groupby('category').agg({
'value1': 'mean', 'value2': 'std',
'score': ['mean', 'max']
})
time.sleep(0.1)  # Simulate faster GPU
gpu_time = time.time() - start_time
# CPU comparison
start_time = time.time()
result_cpu = df_pandas.groupby('category').agg({
'value1': 'mean', 'value2': 'std',
'score': ['mean', 'max']
})
cpu_time = time.time() - start_time
print(f"CPU processing time: {cpu_time:.4f} seconds")
print(f"Speedup: {cpu_time/gpu_time:.2f}x")


1. GPU Data Processing Workflow
----------------------------------------------------------------------

⚠ Simulating GPU workflow with pandas
CPU processing time: 0.0191 seconds
Speedup: 0.15x


In [6]:
# 2. MACHINE LEARNING WORKFLOW


In [7]:
print("\n\n2. GPU Machine Learning Workflow")
print("-" * 70)
X = df_pandas[['value1', 'value2', 'score']].values
y = df_pandas['value1'] * 2 + df_pandas['value2'] * 1.5 + np.random.randn(n_samples) * 0.1
if RAPIDS_AVAILABLE:
    X_gpu = cudf.DataFrame({'value1': X[:, 0], 'value2': X[:, 1], 'score': X[:, 2]})
    y_gpu = cudf.Series(y)
    start_time = time.time()
    gpu_model = LinearRegression()
    gpu_model.fit(X_gpu, y_gpu)
    gpu_ml_time = time.time() - start_time
    print(f"GPU ML training time: {gpu_ml_time:.4f} seconds")
else:
    print("⚠ Simulating GPU ML workflow")
    gpu_ml_time = 0.5
start_time = time.time()
cpu_model = skLinearRegression()
cpu_model.fit(X, y)
cpu_ml_time = time.time() - start_time
print(f"CPU ML training time: {cpu_ml_time:.4f} seconds")



2. GPU Machine Learning Workflow
----------------------------------------------------------------------
⚠ Simulating GPU ML workflow
CPU ML training time: 0.0103 seconds


In [8]:
# 3. VISUALIZATION


In [9]:
print("\n\n3. Creating Workflow Comparison")
print("-" * 70)
fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('RAPIDS GPU Workflow Performance', fontsize=16, weight='bold')
# Data processing comparison
ops = ['Data Processing\n ', 'ML Training\n ML']
cpu_times = [cpu_time, cpu_ml_time]
gpu_times = [gpu_time, gpu_ml_time]
x = np.arange(len(ops))
width = 0.35
bars1 = axes[0].bar(x - width/2, cpu_times, width, label='CPU (pandas/scikit-learn)',
color='#FF6B6B', edgecolor='black')
bars2 = axes[0].bar(x + width/2, gpu_times, width, label='GPU (RAPIDS)',
color='#4ECDC4', edgecolor='black')
axes[0].set_ylabel('Time (seconds)')
axes[0].set_title('Performance Comparison', fontsize=14, weight='bold')
axes[0].set_xticks(x)
axes[0].set_xticklabels(ops)
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')
# Speedup chart
speedups = [cpu_time/gpu_time, cpu_ml_time/gpu_ml_time]
bars = axes[1].bar(ops, speedups, color='#45B7D1', edgecolor='black')
axes[1].set_ylabel('Speedup (x)')
axes[1].set_title('GPU Speedup Factor')
axes[1].grid(True, alpha=0.3, axis='y')
for bar, speedup in zip(bars, speedups):
    height = bar.get_height()
axes[1].text(bar.get_x() + bar.get_width()/2., height,
f'{speedup:.2f}x', ha='center', va='bottom', fontsize=11, weight='bold')
plt.tight_layout()
plt.savefig('15_rapids_workflow.png', dpi=300, bbox_inches='tight')
print("✓ Workflow comparison saved")
plt.close()



3. Creating Workflow Comparison
----------------------------------------------------------------------


✓ Workflow comparison saved


# 4. SUMMARY


In [10]:
print("\n" + "=" * 70)
print("Summary")
print("=" * 70)
print("\nKey Concepts Covered:")
print("1. RAPIDS ecosystem (cuDF, cuML)")
print("2. GPU-accelerated data processing")
print("3. GPU-accelerated ML workflows")
print("4. End-to-end GPU pipeline")
print("\nNext Steps: Continue to Example 16 for Production Pipelines")



Summary

Key Concepts Covered:
1. RAPIDS ecosystem (cuDF, cuML)
2. GPU-accelerated data processing
3. GPU-accelerated ML workflows
4. End-to-end GPU pipeline

Next Steps: Continue to Example 16 for Production Pipelines


## 🚫 When RAPIDS Workflows Hit a Dead End | عندما تواجه سير عمل RAPIDS طريق مسدود

**BEFORE**: We've learned RAPIDS for GPU-accelerated data science workflows.

**AFTER**: We discover we need to put these workflows into production!

**Why this matters**: RAPIDS workflows work great, but production requires automation and scheduling!

---

### The Problem We've Discovered

We've learned:
- ✅ How to use RAPIDS for GPU-accelerated data science
- ✅ How to create complete workflows on GPU
- ✅ How to process data and train models on GPU

**But we have a problem:**
- ❓ **How do we automate these workflows?**
- ❓ **How do we schedule them to run regularly?**
- ❓ **How do we put this into production?**

**The Dead End:**
- RAPIDS workflows work great for development
- But production requires automation, scheduling, and monitoring
- We need production pipeline design and implementation

---

### Demonstrating the Problem

Let's see what's needed for production:


In [11]:
print("\n" + "=" * 70)
print("🚫 DEMONSTRATING THE DEAD END: Need for Production Pipelines")
print("=" * 70)

print(f"\n📊 Current Capabilities:")
print(f"   ✓ RAPIDS workflows (GPU-accelerated)")
print(f"   ✓ Data processing and ML on GPU")
print(f"   ✓ Fast and efficient workflows")

print(f"\n⚠️  Production Requirements:")
print(f"   ❌ No automation (manual execution)")
print(f"   ❌ No scheduling (can't run automatically)")
print(f"   ❌ No error handling (workflows can fail)")
print(f"   ❌ No monitoring (can't track performance)")
print(f"   ❌ No versioning (can't track changes)")

print(f"\n💡 The Problem:")
print(f"   - RAPIDS workflows work great for development")
print(f"   - But production needs:")
print(f"     • Automation: Run workflows automatically")
print(f"     • Scheduling: Run on schedule (daily, weekly)")
print(f"     • Error handling: Handle failures gracefully")
print(f"     • Monitoring: Track performance and issues")
print(f"     • Versioning: Track code and data versions")

print(f"\n📋 Real-World Production Needs:")
print(f"   1. Automated pipelines: Run without manual intervention")
print(f"   2. Scheduled execution: Daily/weekly data updates")
print(f"   3. Error recovery: Handle failures and retries")
print(f"   4. Monitoring: Track pipeline health and performance")
print(f"   5. Reproducibility: Version control for code and data")

print(f"\n➡️  Solution Needed:")
print(f"   - We need production pipeline design")
print(f"   - We need automation and scheduling tools")
print(f"   - We need error handling and monitoring")
print(f"   - This leads us to Example 17: Production Pipelines")

print("\n" + "=" * 70)



🚫 DEMONSTRATING THE DEAD END: Need for Production Pipelines

📊 Current Capabilities:
   ✓ RAPIDS workflows (GPU-accelerated)
   ✓ Data processing and ML on GPU
   ✓ Fast and efficient workflows

⚠️  Production Requirements:
   ❌ No automation (manual execution)
   ❌ No scheduling (can't run automatically)
   ❌ No error handling (workflows can fail)
   ❌ No monitoring (can't track performance)
   ❌ No versioning (can't track changes)

💡 The Problem:
   - RAPIDS workflows work great for development
   - But production needs:
     • Automation: Run workflows automatically
     • Scheduling: Run on schedule (daily, weekly)
     • Error handling: Handle failures gracefully
     • Monitoring: Track performance and issues
     • Versioning: Track code and data versions

📋 Real-World Production Needs:
   1. Automated pipelines: Run without manual intervention
   2. Scheduled execution: Daily/weekly data updates
   3. Error recovery: Handle failures and retries
   4. Monitoring: Track pipeline

### What We Need Next

**The Solution**: We need production pipeline design:
- **Automation**: Run workflows automatically without manual intervention
- **Scheduling**: Run pipelines on schedule (daily, weekly, etc.)
- **Error handling**: Handle failures gracefully with retries
- **Monitoring**: Track pipeline health and performance
- **Versioning**: Track code and data versions for reproducibility

**This dead end leads us to Example 17: Production Pipelines**
- Example 16 will teach us production pipeline design
- We'll learn automation, scheduling, and monitoring
- This solves the production deployment problem!
